# Tests for Eevent Producer

This notebook will contain some tests to assert that transactional events are produced with the expected patterns, and validate refinements made.

In [ ]:
## Non-Uniform Weights of Merchant Categories



In [1]:
# Non-Uniform Weights For Payment Method

import pulse.properties as properties
from pulse.darooghe_pulse import generate_transaction_event
from collections import Counter

NUM_SAMPLES = 50000
TOLERANCE = 0.05

counts = Counter()
for _ in range(NUM_SAMPLES):
    ev = generate_transaction_event()
    counts[ev["payment_method"]] += 1

total = sum(counts.values())

for method, target_weight in properties.PAYMENT_METHOD_WEIGHTS.items():
    observed = counts[method] / total
    assert abs(observed - target_weight) <= TOLERANCE, (
        f"Method {method}: observed {observed:.3f}, target {target_weight:.3f}"
    )

print("Payment method distribution test passed.")


Payment method distribution test passed.
